介绍如何用 Python 语言实现基于VGG19的图像分类
复用第2章实验中的全连接层、ReLU 激活函数层 等基本单元，在此基础上加入新的基本单元如卷积层、最大池化层。

In [1]:
#广播运算
import numpy as np
x = np.array(range(12)).reshape(3,4)


In [2]:
x1=x + 1

In [3]:
x2 = np.array([1,2,3,4])
x3=x + x2

基于三层神经网络实现手写数字分类

DataSet:

In [5]:
def load_mnist(self,file_dir,is_images = 'True'):
    bin_file = open(file_dir,'rb')
    bin_data = bin_file.read()
    bin_file.close()
    if is_images: #read the image data 
        fmt_header = '>iiii'
        magic,num_images,num_rows,num_cols = struct.unpack_from(fmt_header,bin_data,0)
    else: #read label data
        fmt_header = '>ii'
        magic,num_images = struct.unpack_from(fmt_header,bin_data,0)
        num_rows,num_cols = 1 , 1 
    data_size = num_images * num_rows * num_cols
    mat_data = struct,unpack_from('>' + str(data_size) + 'B',bin_data,struct.calcsize(fmt_header))
    mat_data = np.reshape(map_data,[num_images,num_row * num_cols])
    return mat_data

子数据集的读取和预处理

In [ ]:
def load_data(self):
    # TODO: 调用函数 load_mnist 读取和预处理 MNIST 中训练数据和测试数据的图像和标记
    print('Loading MNIST data from files...（使用了load_mnist函数）')
    train_images = self.load_mnist(os.path.join(MNIST_DIR,TRAIN_DATA),True)
    train_labels = self.load_mnist(os.path.join(MNIST_DIR,TRAIN_lABEL),False)
    
    test_images = self.load_mnist(os.path.join(MNIST_DIR, TEST_DATA), True)
    test_labels = self.load_mnist(os.path.join(MNIST_DIR, TEST_LABEL), False)

    self.train_data = np.append(train_images, train_labels, axis=1)
    self.test_data = np.append(test_images, test_labels, axis=1)

In [ ]:
def shuffle_data(self):
    print('Randomly shuffle MNIST data...')
    np.random.shuffle(self.train_data)

:T: X :T:   the usage of axis in append 

axis = 0 时，数据加在列方向

In [6]:
import numpy as np
aa= np.zeros((3,3))
bb=np.ones((3,3))
c = np.append(aa,bb,axis = 0)
print(c)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


axis = 1 时，数据加在行反向

In [8]:
import numpy as np
aa= np.zeros((3,3))
bb=np.ones((3,3))
c = np.append(aa,bb,axis = 1)
print(c)

[[0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]]


不定义axis时，数据为一维

In [3]:
import numpy as np
aa= np.zeros((3,8))
bb=np.ones((3,1))
c = np.append(aa,bb)
print(c)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1.]


通常同类型的层用一个类来定义，类的成员函数通常包括层的初始化、参数的初 始化、前向传播计算、反向传播计算、参数的更新、参数的加载和保存等

In [ ]:
class FullyConnectedLayer(object):

    def __init__(self, num_input, num_output):  # 全连接层初始化
        self.num_input = num_input
        self.num_output = num_output
        print('\tFully connected layer with input %d, output %d.' % (self.num_input, self.num_output))
    
    def init_param(self, std=0.01):  # 参数初始化
        self.weight = np.random.normal(loc=0.0, scale=std, size=(self.num_input, self.num_output))
        self.bias = np.zeros([1, self.num_output])

    def forward(self, input):  # 前向传播计算
        start_time = time.time()
        print('\t开始记时:')
        self.input = input
        # TODO：全连接层的前向传播，计算输出结果
        #Y = XW + b (2.3)
        self.output = np.dot(self.input, self.weight) + self.bias
        return self.output      
#np.dot可以支持矩阵相乘。
##所以下面的反向传播使用的是dot，因为我们通常使用梯度下降法，
##来进行反向梯度下降。选择若干样本同时计算。
##输入就变成了二维矩阵了。

## input , weight ,bias ; 依次对应：x, w ,b ---OK 

    def backward(self, top_diff):  # 反向传播的计算
        # TODO：全连接层的反向传播，计算参数梯度和本层损失
        #(2.4)self.input.T 是input的转置，top_diff 被称为损失函数对上一层的偏导。
        self.d_weight = np.dot(self.input.T, top_diff)

    #d_weight表示此计算层的权重的梯度；下面的d_bias同理。
    #注意，此时反向传播的下一层是正向传播的上一层。
        
        self.batch_size = top_diff.shape[0]

    #shape的功能是读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度
    ##batch_size 为下面的d_bias的维度做个扩展，一般来说与广播计算类似
        
        self.d_bias = np.dot(np.ones(shape=(1,self.batch_size)),top_diff)
        
        bottom_diff = np.dot(top_diff, self.weight.T)
        return bottom_diff

    def update_param(self, lr):  # 参数更新
        # TODO：对全连接层参数利用参数进行更新

    ##lr学习率，确定每次更新时的幅度。
        
        self.weight = self.weight - lr * self.d_weight
        self.bias = self.bias - lr * self.d_bias
    def load_param(self, weight, bias):  # 参数加载
        assert self.weight.shape == weight.shape
        assert self.bias.shape == bias.shape
        self.weight = weight
        self.bias = bias
    def save_param(self):  # 参数保存
        return self.weight, self.bias